In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import folium
from folium.plugins import TimestampedGeoJson
from folium.plugins import HeatMap

In [10]:
expl_df=pd.read_csv('../data/cleaned_data.csv')

In [11]:
data = expl_df[['latitude', 'longitude']].head(10).values.tolist()

# Create a map centered at the mean coordinates with an increased zoom level
mymap = folium.Map(location=[expl_df['latitude'].mean(), expl_df['longitude'].mean()], zoom_start=12)

# Add markers to the map based on the coordinates
for index, row in expl_df.head(10).iterrows():
    # Get the sub_event_type value for the current row
    sub_event_type = row['sub_event_type']
    
    # Create popup text that includes the sub_event_type value
    popup_text = 'Sub-event type: {}'.format(sub_event_type)
    
    # Add a marker with the popup_text
    folium.Marker([row['latitude'], row['longitude']], popup=popup_text).add_to(mymap)

# Save the map
mymap.save('../charts/map.html')


In [12]:
# Group the data by location and calculate the total number of fatalities for each location
location_fatalities = expl_df[expl_df['fatalities'] > 0].groupby(['latitude', 'longitude', 'location'])['fatalities'].sum().reset_index()

# Create a map
mymap = folium.Map(location=[location_fatalities['latitude'].mean(), location_fatalities['longitude'].mean()], zoom_start=7)

# Add markers to the map
for index, row in location_fatalities.iterrows():
    # Calculate the marker size based on a logarithmic scale to reduce size differences
    radius = np.log(row['fatalities'] + 1) * 5
    tooltip_text = f"Location: {row['location']}, Fatalities: {row['fatalities']}"
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=radius/1.5, color='red', fill=True, tooltip=tooltip_text).add_to(mymap)

# Save the map
mymap.save('../charts/fatalities_map.html')






In [13]:



# Convert the event_date column to datetime type
expl_df['event_date'] = pd.to_datetime(expl_df['event_date'])

# Group the data by month and location, and calculate the total number of fatalities for each location
monthly_location_fatalities = expl_df[expl_df['fatalities'] > 0].groupby(['latitude', 'longitude', 'location', pd.Grouper(key='event_date', freq='M')])['fatalities'].sum().reset_index()

# Convert the data into a format suitable for TimestampedGeoJson
features = []
for index, row in monthly_location_fatalities.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'time': row['event_date'].date().__str__(),
            'popup': f"Location: {row['location']}, Fatalities: {row['fatalities']}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'red',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': np.log(row['fatalities'] + 1) * 3,  # Reduce the multiplier
            }
        }
    }
    features.append(feature)

# Create a map
mymap = folium.Map(location=[monthly_location_fatalities['latitude'].mean(), monthly_location_fatalities['longitude'].mean()], zoom_start=7)

# Add TimestampedGeoJson to the map
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1M',
    add_last_point=True,
).add_to(mymap)

# Save the map
mymap.save('../charts/monthly_fatalities_animation.html')



In [14]:
# Filter the data by event types
shelling_data = expl_df[expl_df['sub_event_type'] == 'Shelling/artillery/missile attack']
air_strike_data = expl_df[expl_df['sub_event_type'] == 'Air/drone strike']
explosive_data = expl_df[expl_df['sub_event_type'] == 'Remote explosive/landmine/IED']

# Filter the data by event types
shelling_data = expl_df[expl_df['sub_event_type'] == 'Shelling/artillery/missile attack']
air_strike_data = expl_df[expl_df['sub_event_type'] == 'Air/drone strike']
explosive_data = expl_df[expl_df['sub_event_type'] == 'Remote explosive/landmine/IED']

# Group the data by unique location values
shelling_locations = shelling_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()
air_strike_locations = air_strike_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()
explosive_locations = explosive_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()

# Create a map centered at the mean coordinates
mymap = folium.Map(location=[expl_df['latitude'].mean(), expl_df['longitude'].mean()], zoom_start=7)

# Add circles to the map for each event type with different colors and tooltips
for data, color, event_type in zip([shelling_locations, air_strike_locations, explosive_locations], ['red', 'blue', 'green'], ['Shelling/artillery/missile attack', 'Air/drone strike', 'Remote explosive/landmine/IED']):
    for index, row in data.iterrows():
        folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=5, color=color, fill=True, tooltip=f"Location: {row['location']}\nEvent Type: {event_type}").add_to(mymap)

# Add a legend
legend_html = """
     <div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 80px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:11px;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; Shelling/Artillery/Missile Attack &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i><br>
     &nbsp; Air/Drone Strike &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i><br>
     &nbsp; Remote Explosive/Landmine/IED &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i><br>
      </div>
     """

mymap.get_root().html.add_child(folium.Element(legend_html))

# Save the map
mymap.save('../charts/events_map.html')



In [15]:


df = expl_df.copy()

# Filter by necessary columns
filtered_df = df[['latitude', 'longitude', 'event_type']]  # Select the necessary columns for analysis

# Group by coordinates and event type
grouped_df = filtered_df.groupby(['latitude', 'longitude', 'event_type']).size().reset_index(name='count')

# Create a map
m = folium.Map(location=[filtered_df['latitude'].mean(), filtered_df['longitude'].mean()], zoom_start=5)

# Add a heatmap
heat_data = [[row['latitude'], row['longitude'], row['count']] for index, row in grouped_df.iterrows()]
HeatMap(heat_data, radius=10).add_to(m)

# Save the map
m.save('../charts/heatmap.html')
